In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as pltim
from PIL import Image as im
import numpy as np
import os 
import tensorflow as tf

In [2]:
a = np.zeros((2, 2))
print(a)
b = a.reshape((2, 2, 1))
print(b)



[[ 0.  0.]
 [ 0.  0.]]
[[[ 0.]
  [ 0.]]

 [[ 0.]
  [ 0.]]]


In [3]:
#====Trials
img = pltim.imread("Ellipse_TS/ellipse1.bmp", format="bmp")
print(type(img))
print(img.shape)

img2 = im.open("Ellipse_TS/ellipse1.bmp")
# imgplt2 = plt.pyplot.imshow(img2)
# print(img2)
img2.show()
print(img2.size)
img2.thumbnail((28,28))
print(img2.size)
img2.show()
gray_img = img2.convert('L')
gray_img.show()
# print(gray_img.shape)


# convtonp =  np.ndarray([28, 28, 1])
convtonp = np.array(gray_img)

imgF = img.flatten() 
print(type(img2))
print(type(gray_img))
print(type(convtonp))
print(convtonp.shape)
c = convtonp.reshape((28, 28, 1))
print(c.shape)
# print(c)




<class 'numpy.ndarray'>
(256, 256, 4)
(256, 256)
(28, 28)
<class 'PIL.BmpImagePlugin.BmpImageFile'>
<class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
(28, 28)
(28, 28, 1)


In [4]:
#reading images and data adjustment
all_photos =  np.ndarray([255, 28, 28, 1])#85*3, photo dims
all_photos_names =  []
print(all_photos.shape)

#reading Lines
for i, filename in enumerate(os.listdir("Line_TS")):   
    tmp = im.open("Line_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)

#reading Ellipse
for i, filename in enumerate(os.listdir("Ellipse_TS")):
    tmp = im.open("Ellipse_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i+85] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)
    
#reading Diamonds
for i, filename in enumerate(os.listdir("Diamond_TS")):
    tmp = im.open("Diamond_TS/" + filename)
    tmp.thumbnail((28,28))
    tmp2 = np.array(tmp.convert('L'))
    all_photos[i+170] = tmp2.reshape((28,28,1))
    all_photos_names.append(filename)

print(len(all_photos_names))
img2 = im.open("Line_TS/" + all_photos_names[0])
img2.show()
print(all_photos[0].shape)
print(type(all_photos[0]))

img2 = im.open("Ellipse_TS/" + all_photos_names[85])
img2.show()
print(all_photos[85].shape)
print(type(all_photos[85]))

img2 = im.open("Diamond_TS/" + all_photos_names[170])
img2.show()  
print(all_photos[170].shape)
print(type(all_photos[170]))

(255, 28, 28, 1)
255
(28, 28, 1)
<class 'numpy.ndarray'>
(28, 28, 1)
<class 'numpy.ndarray'>
(28, 28, 1)
<class 'numpy.ndarray'>


In [5]:
#flatten them
all_photosF = np.ndarray([255, 784])#255, 256*256*4
for j, imgg in enumerate(all_photos):
    all_photosF[j] = (imgg.flatten())
    
print(len(all_photosF))

255


In [8]:
#Debugginig
print(type(all_photosF[5]))
print(all_photosF[5].shape)
# print(all_photosF[8])


<class 'numpy.ndarray'>
(784,)


In [10]:
#labels(ground_truth) as one hot key
all_gt =  np.zeros([255, 3])

for i in range(255):
    if i > 169:
        all_gt[i][2] = 1
    elif i > 84:
        all_gt[i][1] = 1
    else:
        all_gt[i][0] = 1

# print(all_gt[0:])

In [11]:
#Training data(210) 
X_tr = np.ndarray([210,784])
y_tr = np.ndarray([210,3])

X_tr[0:70] = all_photosF[0:70] #first 70 lines
X_tr[70:140] = all_photosF[85:155] #first 70 ellipses
X_tr[140:210] = all_photosF[170:240] #first 70 diamonds

y_tr[0:70] = all_gt[0:70]
y_tr[70:140] = all_gt[85:155]
y_tr[140:210] = all_gt[170:240]

print(y_tr.shape)
# print(y_tr[0:210])

(210, 3)


In [12]:
# Validation data(45)
X_v = np.ndarray([45,784])
y_v = np.ndarray([45,3])

X_v[0:15] = all_photosF[70:85]
X_v[15:30] = all_photosF[155:170]
X_v[30:45] = all_photosF[240:255]

y_v[0:15] = all_gt[70:85]
y_v[15:30] = all_gt[155:170]
y_v[30:45] = all_gt[240:255]

print(y_v.shape)
# print(y_v[0:])

(45, 3)


In [13]:
#building the model
#inputs
x = tf.placeholder(tf.float32, shape=[None, 784])

#Paramaters
W = tf.Variable(tf.zeros([784,3]))
b = tf.Variable(tf.zeros([3]))

#output
y = tf.matmul(x,W) + b
y_softmax = tf.nn.softmax(y)

#Gorund truth
y_ = tf.placeholder(tf.float32, [None, 3])

print(x.shape)
print(y_.shape)

(?, 784)
(?, 3)


In [14]:
#Compute Loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

#Update to improve this loss margin
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#Now We can launch the model
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [15]:
#train 1000 step
from datetime import datetime
start=datetime.now()

for _ in range(10000):
#     batch_xs, batch_ys = X_tr[i:i+3], y_tr[i:i+3]
#     sess.run(train_step)
    sess.run(train_step, feed_dict={x: X_tr, y_: y_tr})

print("Ellapsed Time: ")
print (datetime.now()-start)

Ellapsed Time: 
0:00:20.825454


In [16]:
#Evaluating
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: X_v, y_: y_v})) #!!!

0.844445


In [ ]:
#converting it into CNN 
#Data augmentation
#Where are my weights now?! 
#batches
#check_ponits(every specific time ?!)
#Save, Restore
#Analysis (model summary and tensorboard)
#Converting to cnn

In [17]:
#Saving and creating checkpoints
saver = tf.train.Saver()
save_path = saver.save(sess, "chkpts/model_3n.ckpt")

In [ ]:
#Restoring variables
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, "chkpts/model_1n.ckpt")
    print("Model restored.")

In [ ]:
# momken ba2a ba3d ama a-restore el model a3mel test ezai ?!
print('test accuracy %g' % accuracy.eval(feed_dict={x: X_v, y_: y_v}))

In [ ]:
from IPython.display import Audio
sound_file = './sound/finish.mp3'
Audio(url=sound_file, autoplay=True)